## 필요한 라이브러리 설치 임포트

In [1]:
!pip install konlpy --upgrade

In [15]:
import requests
from bs4 import BeautifulSoup
from transformers import pipeline
from konlpy.tag import Okt
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
import json
from IPython.display import Image, display, Markdown

## 네이버 뉴스 크롤링

In [3]:
def get_soup_obj(news_link):
  headers = {'User-Agent': 'Mozilla/5.0'}
  response = requests.get(news_link, headers=headers)
  soup = BeautifulSoup(response.text, 'html.parser')

  return soup

In [4]:
def get_top3_news_info(sec, sid):
  default_img = "https://search.naver.com/search.naver?where=image$sm=tab_jum$query=naver#"
  sec_url = "https://news.naver.com/section/%s" % sid
  print("section url : ", sec_url)

  news_list3 = []
  soup = get_soup_obj(sec_url)

  lis3 = soup.find_all("li", class_="sa_item _SECTION_HEADLINE", limit=3)
  for li in lis3:
    title_tag = li.find('strong', class_="sa_text_strong")
    a_tag = li.select_one('a.sa_text_title')
    media_tag = li.find('div', class_='sa_text_info_left')
    img_tag = li.find('img')


    news_info = {
        'title': title_tag.get_text(strip=True) if title_tag else None,
        'media_com': media_tag.get_text(strip=True) if media_tag else None,
        'news_link': a_tag['href'] if a_tag and 'href' in a_tag.attrs else None,
        'img_src': (img_tag.get('data-src') or img_tag.get('src')) if img_tag else None
    }
    news_list3.append(news_info)

  return news_list3

In [5]:
def get_article_contents(news_link):
    soup = get_soup_obj(news_link)

    # 다양한 구조 대응
    article = soup.find('article', id='dic_area') \
           or soup.find('div', id='newsct_article') \
           or soup.find('div', class_='article_body')

    if not article:
        return "본문을 찾을 수 없습니다."

    content = article.get_text(separator='\n', strip=True)

    return content

In [6]:
summarizer = pipeline("summarization", model="csebuetnlp/mT5_multilingual_XLSum")

def naver_news_top3():
  news_dic = dict()
  sections = ['pol', 'eco', 'soc', 'cul', 'wor', 'sci']
  section_ids = ['100', '101', '102', '103', '104', '105']

  for sec, sid in zip(sections, section_ids):
    news_info = get_top3_news_info(sec, sid)

    for news in news_info:
      news_link = news['news_link']
      news_contents = get_article_contents(news_link)
      news['news_contents'] = news_contents

      try:
        summary_output = summarizer(news_info['news_contents'], max_length=128, min_length=30, do_sample=False)
        snews_contents = summary_output[0]['summary_text']
      except Exception as e:
        sentences = news_contents.split('.')
        if len(sentences) > 3:
          snews_contents = '.'.join(sentences[:3])+'.'
        else:
          snews_contents = news_contents

      news['snews_contents'] = snews_contents

    news_dic[sec] = news_info

  return news_dic

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/pytho

In [7]:
news_dic = naver_news_top3()
news_dic

section url :  https://news.naver.com/section/100
section url :  https://news.naver.com/section/101
section url :  https://news.naver.com/section/102
section url :  https://news.naver.com/section/103
section url :  https://news.naver.com/section/104
section url :  https://news.naver.com/section/105


{'pol': [{'title': '법사위, 與 주도 노란봉투법·농업2법·방송3법 의결..4일 본회의로',
   'media_com': '파이낸셜뉴스',
   'news_link': 'https://n.news.naver.com/mnews/article/014/0005385916',
   'img_src': 'https://mimgnews.pstatic.net/image/origin/014/2025/08/01/5385916.jpg?type=nf220_150',
   'news_contents': '22일 오후 서울 여의도 국회에서 열린 법제사법위원회 제427회국회(임시회) 제4차 전체회의. 사진=뉴스1\n[파이낸셜뉴스] 국회 법제사법위원회는 1일 더불어민주당 주도로 쟁점법안들을 의결했다.\n법사위는 이날 전체회의에서 쟁점법안인 노란봉투법과 농업2법, 방송3법을 처리했다. 여야 합의를 이루지 못했지만 민주당이 시급하다는 판단으로 밀어붙였다. 오는 4일 국회 본회의에 상정될 예정이다.\n먼저 노란봉투법은 파업 노동자에 대한 손해배상 청구를 제한하고, 하청업체 노동자에게 원청을 상대로 한 교섭권을 부여하는 게 골자이다. 경제계에서는 지나친 노동쟁의와 노사교섭으로 기업활동에 지장을 초래할 수 있다고 우려하고 있다.\n농업2법은 정부가 과잉 생산된 쌀을 의무적으로 매입하도록 하는 양곡관리법 개정안, 주요 농산물 시장가격이 기준가격 미만으로 하락할 경우 생산자에게 차액을 지급하는 농수산물 유통 및 가격안정법 개정안 등이다. 전임 윤석열 정부가 재정부담이 크다는 이유로 재의요구권(거부권)을 행사해 폐기됐다가 민주당이 다시 추진했다.\n방송3법은 방송법·방송문화진흥회법·한국교육방송공사법을 개정해 공영방송 지배구조를 개선하려는 취지이다. KBS, MBC, EBS 이사회를 확대하고 추천 주체를 다양화하고, 공영방송사 사장 임명에 사장후보추천위원회 추천을 거치도록 하는 내용이다.\n법사위는 이날 오후 전체회의를 속개하고 쟁점법안인 2차 상법 개정안도 처리할 예정이다. 이번 상법 

## 댓글 스크래핑

In [8]:
def extract_press_article_id(news_link):
    match = re.search(r'oid=(\d+)&aid=(\d+)', news_link)
    if match:
        return match.groups()

    match = re.search(r'/article/(\d{3})/(\d+)', news_link)

    if match:
        return match.groups()

    return None, None


def get_comment_count(press_id, article_id):
    url = "https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json"
    params = {
        'ticket': 'news',
        'templateId': 'default',
        'pool': 'cbox5',
        'lang': 'ko',
        'country': 'KR',
        'objectId': f"news{press_id},{article_id}",
        'pageSize': 1,
        'indexSize': 10
    }
    headers = {
        'Referer': f'https://news.naver.com/main/read.naver?oid={press_id}&aid={article_id}',
        'User-Agent': 'Mozilla/5.0'
    }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        try:
            json_str = response.text[response.text.find('(')+1:-2]
            data = json.loads(json_str)
            return data['result']['count']['total']
        except Exception as e:
            print(f"[❌ JSON 파싱 오류] {press_id}, {article_id}: {e}")
            return None
    else:
        print(f"[❌ 요청 실패] status_code={response.status_code}, press_id={press_id}, article_id={article_id}")
        return None

In [9]:
def count_comments(news_dic, sec):
  for article in articles:
      news_link = article.get('news_link')
      press_id, article_id = extract_press_article_id(news_link)
      if press_id and article_id:
          count_comment = get_comment_count(press_id, article_id)
      else:
          count_comment = None
  return count_comment

In [10]:
def get_comments_only(news_link):
    press_id, article_id = extract_press_article_id(news_link)
    if not press_id or not article_id:
        return []

    object_id = f"news{press_id},{article_id}"
    referer_url = f"https://news.naver.com/main/read.naver?oid={press_id}&aid={article_id}"
    url = "https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json"
    headers = {
        'Referer': referer_url,
        'User-Agent': 'Mozilla/5.0'
    }

    comments = []
    page = 1
    while True:
        params = {
            'ticket': 'news',
            'templateId': 'default',
            'pool': 'cbox5',
            'lang': 'ko',
            'country': 'KR',
            'objectId': object_id,
            'pageSize': 100,
            'indexSize': 10,
            'page': page
        }

        response = requests.get(url, headers=headers, params=params)
        if response.status_code != 200:
            break

        try:
            json_str = response.text[response.text.find('(')+1:-2]
            data = json.loads(json_str)
            comment_list = data['result'].get('commentList', [])
            if not comment_list:
                break

            for c in comment_list:
                comments.append(c['contents'])

            page += 1
            time.sleep(0.2)

        except:
            break

    return comments

In [11]:
def collect_all_comments_only(news_dic, sec):
  all_comments_list = []
  for article in articles:
      news_link = article.get('news_link')
      comments = get_comments_only(news_link)
      all_comments_list.extend(comments)
  return pd.DataFrame({'comment': all_comments_list})

In [12]:
def comment_emotion(df_all_comments, model, tfidvect):
  def text_cleaning(text):
    text = re.sub(r"<.*?>|&[^;]+;", " ", text)
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"[^가-힣a-zA-Z0-9 ]", "", text)
    return text

  def get_pos(x):
    okt = Okt()
    pos = okt.pos(x)
    return ['{}/{}'.format(word, tag) for word, tag in pos if tag not in ['Josa', 'Eomi', 'Punctuation']]

  df_all_comments['cleaned'] = df_all_comments['comment'].apply(text_cleaning)
  df_all_comments['pos'] = df_all_comments['cleaned'].apply(get_pos)

  X = tfidvect.transform(df_all_comments['pos'])

  preds = model.predict(X)

  df_all_comments['pred'] = preds
  df_all_comments['emotion'] = df_all_comments['pred'].apply(
      lambda x: '긍정' if x==1 else('부정' if x==0 else '모호함')
      )

  return df_all_comments[['comment', 'emotion']]

In [13]:
def emotion_ratio(result_df):
  total = len(result_df)

  emotion_ratio = (
      result_df['emotion']
      .value_counts(normalize=True)
      .mul(100)
      .round(2)
      .to_dict()
  )

  return emotion_ratio

In [14]:
def identity_tokenizer(x): return x
def identity_preprocessor(x): return x
model = joblib.load('sentiment_model.pkl')
tfidvect = joblib.load('tfidf_vectorizer.pkl')

sections = ['pol', 'eco', 'soc', 'cul', 'wor', 'sci']

for sec in sections:
    print(f"\n📂 섹션: {sec}")
    articles = news_dic.get(sec, [])

    for i, article in enumerate(articles):
        news_link = article.get('news_link')
        news_title = article.get('title')
        print(f" 📰 기사 {i+1}: {news_title}")

        comments = get_comments_only(news_link)
        comment_count = len(comments)

        if comment_count >= 5:
            print(f" 🥎 댓글 수가 {comment_count}개로 충분합니다. 댓글 분석을 시작합니다.")

            df_all_comments = collect_all_comments_only(news_dic, sec)
            result_df = comment_emotion(df_all_comments, model, tfidvect)

            emotion_ratios = emotion_ratio(result_df)
            for emotion, ratio in emotion_ratios.items():
              print(f"해당 기사에 대해 {emotion} 반응은 {ratio:.2f}%입니다.")


        else:
            print(f" ⚾️ 댓글 수가 {comment_count}개로 부족하여, 시장 반응을 확인하기 어렵습니다.")
        print('\n')


📂 섹션: pol
 📰 기사 1: 법사위, 與 주도 노란봉투법·농업2법·방송3법 의결..4일 본회의로
 🥎 댓글 수가 36개로 충분합니다. 댓글 분석을 시작합니다.
해당 기사에 대해 모호함 반응은 55.80%입니다.
해당 기사에 대해 부정 반응은 37.68%입니다.
해당 기사에 대해 긍정 반응은 6.52%입니다.


 📰 기사 2: 이재명 대통령, 취임 두 달 만 첫 여름 휴가…“거제 저도서 재충전”
 🥎 댓글 수가 99개로 충분합니다. 댓글 분석을 시작합니다.
해당 기사에 대해 모호함 반응은 55.80%입니다.
해당 기사에 대해 부정 반응은 37.68%입니다.
해당 기사에 대해 긍정 반응은 6.52%입니다.


 📰 기사 3: 이재명 대통령, 취임 후 첫 '시·도지사 간담회' 개최 [뉴시스Pic]
 ⚾️ 댓글 수가 3개로 부족하여, 시장 반응을 확인하기 어렵습니다.



📂 섹션: eco
 📰 기사 1: 삼성SDI 2분기 영업손실 4000억 원...희비 엇갈린 K배터리
 ⚾️ 댓글 수가 1개로 부족하여, 시장 반응을 확인하기 어렵습니다.


 📰 기사 2: 정부, ‘전기 저수지’ ESS 사업자 전남·제주 8곳 확정…전력계통 안정성 기대
 ⚾️ 댓글 수가 0개로 부족하여, 시장 반응을 확인하기 어렵습니다.


 📰 기사 3: 삼성 HBM4까지 가세…엔비디아 내년 신제품 겨냥 ‘진검승부’ 시작됐다[비즈360]
 🥎 댓글 수가 11개로 충분합니다. 댓글 분석을 시작합니다.
해당 기사에 대해 모호함 반응은 50.00%입니다.
해당 기사에 대해 부정 반응은 33.33%입니다.
해당 기사에 대해 긍정 반응은 16.67%입니다.



📂 섹션: soc
 📰 기사 1: 약속한 듯 같은 날 3번째, 용인·부산 이어 세종서도…택시 상가 돌진, 1명 부상
 🥎 댓글 수가 56개로 충분합니다. 댓글 분석을 시작합니다.
해당 기사에 대해 모호함 반응은 52.63%입니다.
해당 기사에 대해 부정 반응은 41.05%입니다.
해당 기사에 대해 긍정 반응은 6.32%입니다.


 📰 기

## 섹션 선택 및 헤드라인 기사 추출

In [17]:
sections = ['pol', 'eco', 'soc', 'cul', 'wor', 'sci']
print("사용 가능한 섹션:", sections)

selected_sec = input("검색할 섹션을 입력하세요: ").strip()

if selected_sec not in sections:
    print("❌ 유효하지 않은 섹션입니다. 다시 실행해주세요.")
else:
    print(f"✅ 선택된 섹션: {selected_sec}")
    articles = news_dic.get(sec, [])

    for i, article in enumerate(articles):
        news_link = article.get('news_link')
        news_title = article.get('title')
        img_url = article['img_src']
        summary = article['snews_contents'].replace('\n', '\n\n')
        print(f"📰 기사 {i+1}")
        display(Image(url=img_url))
        display(Markdown(f"### 📰 {article['title']}\n\n{summary}"))

        comments = get_comments_only(news_link)
        comment_count = len(comments)

        if comment_count >= 5:
            print(f" 🥎 댓글 수가 {comment_count}개로 충분합니다. 댓글 분석을 시작합니다.")

            df_all_comments = collect_all_comments_only(news_dic, sec)
            result_df = comment_emotion(df_all_comments, model, tfidvect)

            emotion_ratios = emotion_ratio(result_df)
            for emotion, ratio in emotion_ratios.items():
              print(f"해당 기사에 대해 {emotion} 반응은 {ratio:.2f}%입니다.")


        else:
            print(f" ⚾️ 댓글 수가 {comment_count}개로 부족하여, 시장 반응을 확인하기 어렵습니다.")
        print('\n')

사용 가능한 섹션: ['pol', 'eco', 'soc', 'cul', 'wor', 'sci']
검색할 섹션을 입력하세요: pol
✅ 선택된 섹션: pol
📰 기사 1


### 📰 “감자의 조상은 토마토였다?”…진화가 만든 완벽한 작물

감자는 약 900만 년 전, 토마토와 에투베로숨이라는 야생 식물이 교배해 탄생한 것으로 밝혀졌다. 이번 연구는 감자의 유전체 분석을 통해 진화의 기원을 최초로 규명했다. (사진=게티이미지)

감자가 약 900만 년 전, 토마토와 야생 식물의 교배로 탄생했다는 연구 결과가 나왔다.

 🥎 댓글 수가 13개로 충분합니다. 댓글 분석을 시작합니다.
해당 기사에 대해 모호함 반응은 60.71%입니다.
해당 기사에 대해 부정 반응은 30.36%입니다.
해당 기사에 대해 긍정 반응은 8.93%입니다.


📰 기사 2


### 📰 애플 실적 호조, 중국 매출액도 늘었다…시간외 주가 강세

뉴욕 애플 스토어

애플이 7월31일(현지시간) 장 마감 후 시장 전망치를 웃도는 분기 실적을 발표했다. 애플 주가는 시간외거래에서 약 3% 오르고 있다.

애플은 이날 장 마감 후 2025 회계연도 3분기(올 4~6월) 주당순이익(EPS)이 1.

 ⚾️ 댓글 수가 0개로 부족하여, 시장 반응을 확인하기 어렵습니다.


📰 기사 3


### 📰 미국서 삼성 '폴드7' 대박 조짐 보인다…여성들 난리

갤Z폴드·플립7 美 사전 판매 흥행

Z폴드7, Z 시리즈 중 '최다' 판매

Z폴드·플립7, 판매량 25% 증가

"미국 여성들, Z폴드 수요 늘어"

영상=제리릭에브리띵 유튜브 채널 영상 갈무리

삼성전자가 미국 스마트폰 시장을 흔들고 있다. 올 2분기 현지 시장 점유율을 끌어올리면서 애플과의 격차를 좁힌 데 이어 최근 출시한 초슬림 폴더블 스마트폰 갤럭시Z폴드·플립7이 역대급 판매량을 기록한 것이다. 특히 미국 여성 소비자들 사이에서 Z폴드7 수요가 증가하고 있는 것으로 나타났다.

 🥎 댓글 수가 43개로 충분합니다. 댓글 분석을 시작합니다.
해당 기사에 대해 모호함 반응은 60.71%입니다.
해당 기사에 대해 부정 반응은 30.36%입니다.
해당 기사에 대해 긍정 반응은 8.93%입니다.


